In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import learn
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn import cross_validation
import numpy as np

boston = learn.datasets.load_dataset('boston')
x, y = boston.data, boston.target
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(
x, y, test_size=0.2, random_state=42)
print(x.shape)
print(y.shape)
total_len = X_train.shape[0]
print(x)
print(y)

(506, 13)
(506,)
[[  6.32000000e-03   1.80000000e+01   2.31000000e+00 ...,   1.53000000e+01
    3.96900000e+02   4.98000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.96900000e+02   9.14000000e+00]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00 ...,   1.78000000e+01
    3.92830000e+02   4.03000000e+00]
 ..., 
 [  6.07600000e-02   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.96900000e+02   5.64000000e+00]
 [  1.09590000e-01   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.93450000e+02   6.48000000e+00]
 [  4.74100000e-02   0.00000000e+00   1.19300000e+01 ...,   2.10000000e+01
    3.96900000e+02   7.88000000e+00]]
[ 24.   21.6  34.7  33.4  36.2  28.7  22.9  27.1  16.5  18.9  15.   18.9
  21.7  20.4  18.2  19.9  23.1  17.5  20.2  18.2  13.6  19.6  15.2  14.5
  15.6  13.9  16.6  14.8  18.4  21.   12.7  14.5  13.2  13.1  13.5  18.9
  20.   21.   24.7  30.8  34.9  26.6  25.3  24.7  21.2  19.3  20.   16.6
  14.4  19.

In [19]:
# Parameters
learning_rate = 0.0006
training_epochs = 500
batch_size = 10
display_step = 1
dropout_rate = 0.9
# Network Parameters
n_hidden_1 = 32 # 1st layer number of features
n_hidden_2 = 200 # 2nd layer number of features
n_hidden_3 = 200
n_hidden_4 = 256
n_input = X_train.shape[1]
n_classes = 1

# tf Graph input
x = tf.placeholder("float", [None, 13])
y = tf.placeholder("float", [None])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with RELU activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0, 0.1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0, 0.1)),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3], 0, 0.1)),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_hidden_4, n_classes], 0, 0.1))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], 0, 0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], 0, 0.1)),
    'b3': tf.Variable(tf.random_normal([n_hidden_3], 0, 0.1)),
    'b4': tf.Variable(tf.random_normal([n_hidden_4], 0, 0.1)),
    'out': tf.Variable(tf.random_normal([n_classes], 0, 0.1))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)
tf.transpose(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        print ("num batch:", total_batch)

        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
            print ("[*]----------------------------")
            for i in xrange(3):
                print ("label value:", label_value[i], \
                    "estimated value:", estimate[i])
            print ("[*]============================")

    print ("Optimization Finished!")
    
     
    #tuple(tensor_name.get_shape().as_list())
    
    # Test model
    predicted_vals = sess.run(pred, feed_dict={x: X_test})    
    print(predicted_vals)
    # Calculate accuracy
    accuracy = sess.run(cost, feed_dict={x:X_test, y: Y_test}) 
    print(accuracy)
    

Instructions for updating:
Use `tf.global_variables_initializer` instead.
num batch: 40
Epoch: 0001 cost= 158.376550627
[*]----------------------------
label value: 23.9 estimated value: [ 23.79486847]
label value: 50.0 estimated value: [ 22.3652668]
label value: 22.8 estimated value: [ 23.66727066]
[*]============================
num batch: 40
Epoch: 0002 cost= 94.127221680
[*]----------------------------
label value: 23.9 estimated value: [ 21.80575371]
label value: 50.0 estimated value: [ 20.14458084]
label value: 22.8 estimated value: [ 20.97412491]
[*]============================
num batch: 40
Epoch: 0003 cost= 92.245317388
[*]----------------------------
label value: 23.9 estimated value: [ 21.97220993]
label value: 50.0 estimated value: [ 20.80865669]
label value: 22.8 estimated value: [ 21.40110397]
[*]============================
num batch: 40
Epoch: 0004 cost= 92.732222033
[*]----------------------------
label value: 23.9 estimated value: [ 21.87491798]
label value: 50.0 esti

num batch: 40
Epoch: 0033 cost= 89.365957069
[*]----------------------------
label value: 23.9 estimated value: [ 23.88571739]
label value: 50.0 estimated value: [ 22.60730743]
label value: 22.8 estimated value: [ 22.85772896]
[*]============================
num batch: 40
Epoch: 0034 cost= 89.210496569
[*]----------------------------
label value: 23.9 estimated value: [ 23.98683929]
label value: 50.0 estimated value: [ 22.75728989]
label value: 22.8 estimated value: [ 22.93621063]
[*]============================
num batch: 40
Epoch: 0035 cost= 89.041097355
[*]----------------------------
label value: 23.9 estimated value: [ 24.04655266]
label value: 50.0 estimated value: [ 22.80534554]
label value: 22.8 estimated value: [ 22.96522713]
[*]============================
num batch: 40
Epoch: 0036 cost= 89.069409680
[*]----------------------------
label value: 23.9 estimated value: [ 24.14234734]
label value: 50.0 estimated value: [ 22.99513435]
label value: 22.8 estimated value: [ 23.059152

num batch: 40
Epoch: 0065 cost= 87.834762096
[*]----------------------------
label value: 23.9 estimated value: [ 24.42038536]
label value: 50.0 estimated value: [ 23.63015175]
label value: 22.8 estimated value: [ 23.41465569]
[*]============================
num batch: 40
Epoch: 0066 cost= 87.710643387
[*]----------------------------
label value: 23.9 estimated value: [ 24.47550011]
label value: 50.0 estimated value: [ 23.65943527]
label value: 22.8 estimated value: [ 23.38245392]
[*]============================
num batch: 40
Epoch: 0067 cost= 88.051813221
[*]----------------------------
label value: 23.9 estimated value: [ 23.60954475]
label value: 50.0 estimated value: [ 22.94537163]
label value: 22.8 estimated value: [ 22.53528786]
[*]============================
num batch: 40
Epoch: 0068 cost= 87.564870644
[*]----------------------------
label value: 23.9 estimated value: [ 24.44826508]
label value: 50.0 estimated value: [ 23.60760117]
label value: 22.8 estimated value: [ 23.300247

num batch: 40
Epoch: 0097 cost= 86.832666683
[*]----------------------------
label value: 23.9 estimated value: [ 24.79885483]
label value: 50.0 estimated value: [ 24.06074905]
label value: 22.8 estimated value: [ 23.23474312]
[*]============================
num batch: 40
Epoch: 0098 cost= 86.944918013
[*]----------------------------
label value: 23.9 estimated value: [ 24.77662849]
label value: 50.0 estimated value: [ 24.14214516]
label value: 22.8 estimated value: [ 23.25102615]
[*]============================
num batch: 40
Epoch: 0099 cost= 86.880863094
[*]----------------------------
label value: 23.9 estimated value: [ 25.12865067]
label value: 50.0 estimated value: [ 24.21665382]
label value: 22.8 estimated value: [ 23.41459846]
[*]============================
num batch: 40
Epoch: 0100 cost= 86.967391300
[*]----------------------------
label value: 23.9 estimated value: [ 24.72854805]
label value: 50.0 estimated value: [ 23.96925354]
label value: 22.8 estimated value: [ 23.208473

num batch: 40
Epoch: 0129 cost= 86.069681168
[*]----------------------------
label value: 23.9 estimated value: [ 25.20078659]
label value: 50.0 estimated value: [ 24.4430027]
label value: 22.8 estimated value: [ 23.4655571]
[*]============================
num batch: 40
Epoch: 0130 cost= 86.010342360
[*]----------------------------
label value: 23.9 estimated value: [ 25.28508568]
label value: 50.0 estimated value: [ 24.87822342]
label value: 22.8 estimated value: [ 23.60462952]
[*]============================
num batch: 40
Epoch: 0131 cost= 85.950660229
[*]----------------------------
label value: 23.9 estimated value: [ 25.3307972]
label value: 50.0 estimated value: [ 24.29930305]
label value: 22.8 estimated value: [ 23.51779556]
[*]============================
num batch: 40
Epoch: 0132 cost= 85.892080784
[*]----------------------------
label value: 23.9 estimated value: [ 25.15590668]
label value: 50.0 estimated value: [ 24.62597084]
label value: 22.8 estimated value: [ 23.52448082]

...

num batch: 40
Epoch: 0353 cost= 81.746343732
[*]----------------------------
label value: 23.9 estimated value: [ 26.95630646]
label value: 50.0 estimated value: [ 26.67111969]
label value: 22.8 estimated value: [ 23.80653]
[*]============================
num batch: 40
Epoch: 0354 cost= 81.714101601
[*]----------------------------
label value: 23.9 estimated value: [ 26.10276794]
label value: 50.0 estimated value: [ 26.21518898]
label value: 22.8 estimated value: [ 23.26002502]
[*]============================
num batch: 40
Epoch: 0355 cost= 82.199220777
[*]----------------------------
label value: 23.9 estimated value: [ 25.17926216]
label value: 50.0 estimated value: [ 26.43589211]
label value: 22.8 estimated value: [ 24.01942253]
[*]============================
num batch: 40
Epoch: 0356 cost= 82.081406975
[*]----------------------------
label value: 23.9 estimated value: [ 26.76672173]
label value: 50.0 estimated value: [ 26.26486397]
label value: 22.8 estimated value: [ 24.00381279]

num batch: 40
Epoch: 0385 cost= 81.866081738
[*]----------------------------
label value: 23.9 estimated value: [ 29.02962112]
label value: 50.0 estimated value: [ 26.47549438]
label value: 22.8 estimated value: [ 23.56409836]
[*]============================
num batch: 40
Epoch: 0386 cost= 81.843966317
[*]----------------------------
label value: 23.9 estimated value: [ 28.5825119]
label value: 50.0 estimated value: [ 26.72893906]
label value: 22.8 estimated value: [ 23.63742638]
[*]============================
num batch: 40
Epoch: 0387 cost= 81.784117818
[*]----------------------------
label value: 23.9 estimated value: [ 29.71882248]
label value: 50.0 estimated value: [ 27.05606461]
label value: 22.8 estimated value: [ 24.08647156]
[*]============================
num batch: 40
Epoch: 0388 cost= 81.778487611
[*]----------------------------
label value: 23.9 estimated value: [ 28.32678986]
label value: 50.0 estimated value: [ 26.90897369]
label value: 22.8 estimated value: [ 23.8317394

num batch: 40
Epoch: 0417 cost= 81.760904884
[*]----------------------------
label value: 23.9 estimated value: [ 28.35107613]
label value: 50.0 estimated value: [ 25.19920349]
label value: 22.8 estimated value: [ 23.47271538]
[*]============================
num batch: 40
Epoch: 0418 cost= 81.961290646
[*]----------------------------
label value: 23.9 estimated value: [ 26.76659775]
label value: 50.0 estimated value: [ 26.39074326]
label value: 22.8 estimated value: [ 23.21421051]
[*]============================
num batch: 40
Epoch: 0419 cost= 82.009271932
[*]----------------------------
label value: 23.9 estimated value: [ 27.10695648]
label value: 50.0 estimated value: [ 25.31235886]
label value: 22.8 estimated value: [ 23.18666077]
[*]============================
num batch: 40
Epoch: 0420 cost= 81.772478676
[*]----------------------------
label value: 23.9 estimated value: [ 27.93553734]
label value: 50.0 estimated value: [ 25.17160225]
label value: 22.8 estimated value: [ 23.526636

num batch: 40
Epoch: 0449 cost= 81.315890074
[*]----------------------------
label value: 23.9 estimated value: [ 27.47990799]
label value: 50.0 estimated value: [ 24.71354866]
label value: 22.8 estimated value: [ 24.88405037]
[*]============================
num batch: 40
Epoch: 0450 cost= 81.099562001
[*]----------------------------
label value: 23.9 estimated value: [ 28.06122398]
label value: 50.0 estimated value: [ 24.84998131]
label value: 22.8 estimated value: [ 25.04500008]
[*]============================
num batch: 40
Epoch: 0451 cost= 80.824115705
[*]----------------------------
label value: 23.9 estimated value: [ 28.23226929]
label value: 50.0 estimated value: [ 25.27305984]
label value: 22.8 estimated value: [ 24.85183525]
[*]============================
num batch: 40
Epoch: 0452 cost= 80.940839958
[*]----------------------------
label value: 23.9 estimated value: [ 28.45446014]
label value: 50.0 estimated value: [ 25.09513664]
label value: 22.8 estimated value: [ 23.437276

num batch: 40
Epoch: 0481 cost= 80.934151077
[*]----------------------------
label value: 23.9 estimated value: [ 29.12564087]
label value: 50.0 estimated value: [ 25.98025703]
label value: 22.8 estimated value: [ 23.81114006]
[*]============================
num batch: 40
Epoch: 0482 cost= 81.021079302
[*]----------------------------
label value: 23.9 estimated value: [ 27.727705]
label value: 50.0 estimated value: [ 26.0143013]
label value: 22.8 estimated value: [ 23.88666344]
[*]============================
num batch: 40
Epoch: 0483 cost= 81.115372372
[*]----------------------------
label value: 23.9 estimated value: [ 28.1199646]
label value: 50.0 estimated value: [ 26.94714928]
label value: 22.8 estimated value: [ 23.98768616]
[*]============================
num batch: 40
Epoch: 0484 cost= 80.978606439
[*]----------------------------
label value: 23.9 estimated value: [ 27.03389549]
label value: 50.0 estimated value: [ 27.34851456]
label value: 22.8 estimated value: [ 24.16080475]
